# FastText Inference Process

In this notebook we will analyze the inner workings of a FastText supervised model when making inferences. We will replicate what the trained model does in plain Python so it's easier to understand. The process can be split into the following steps:

1. Getting an embedding for the given sentence.
2. Computing the inferred class given this embedding.

First we train a simple model.

In [ ]:
%%capture
! pip install fasttext
! wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
! head -n 12404 cooking.stackexchange.txt > cooking.train
! tail -n 3000 cooking.stackexchange.txt > cooking.valid

In [ ]:
! ls

cooking.stackexchange.id	cooking.stackexchange.txt  readme.txt
cooking.stackexchange.tar.gz	cooking.train		   sample_data
cooking.stackexchange.tar.gz.1	cooking.valid


In [ ]:
import numpy as np
import fasttext
from typing import List

In [ ]:
LR=1.0 
EPOCH=30 
WORD_NGRAMS=2 
BUCKET=200000
DIM=16
MINN=3
MAXN=6

In [ ]:
model = fasttext.train_supervised(input="cooking.train", lr=LR, epoch=EPOCH, wordNgrams=WORD_NGRAMS, 
                                  bucket=BUCKET, dim=DIM, minn=MINN, maxn=MAXN)

# Get FastText parameters

Now we get the parameters and hyperparameters from the FastText binary model.

In [ ]:
vocabulary = model.get_words()
labels = model.labels
embedding_dim = model.get_dimension()
bucket_size = model.f.getArgs().bucket
minn = model.f.getArgs().minn
maxn = model.f.getArgs().maxn
wordNgrams = model.f.getArgs().wordNgrams
input_matrix = model.get_input_matrix()
output_matrix = model.get_output_matrix()

print("Vocabulary size:", len(vocabulary))
print("Number of labels:", len(labels))
print("Embedding dimension:", embedding_dim)
print("Bucket size:", bucket_size)
print("Minn:", minn)
print("Maxn:", maxn)
print("wordNgrams max length:", wordNgrams)
print("Input matrix shape:", input_matrix.shape)
print("Output matrix shape:", output_matrix.shape)

Vocabulary size: 14543
Number of labels: 735
Embedding dimension: 16
Bucket size: 200000
Minn: 3
Maxn: 6
wordNgrams max length: 2
Input matrix shape: (214543, 16)
Output matrix shape: (735, 16)


# 1. Compute full word vectors

Full word vectors are stored in the first chunk of the input_matrix. These are words that show up in the training data at least **minCount** times. If there are no character n-grams then this vector will be the final word vector. If there are character n-grams then the final word vector is the mean of the word vector and the subword vectors.

In [ ]:
word = "meatballs"
idx_a = model.get_words().index(word)
idx_a

957

In [ ]:
idx_b = model.get_word_id(word)
idx_b

957

In [ ]:
input_matrix[idx_a]

array([ 0.26651105,  0.02829988, -0.18690547,  0.31578544, -0.4148955 ,
        0.71081346,  0.8592059 , -0.30728486, -0.15301947, -0.59633934,
        0.45763463, -0.22829835, -0.14477722,  0.29369563, -0.02462302,
        0.21090278], dtype=float32)

# 2. Compute all possible subwords

FastText adds the "Beginning of Word" and "End of Word" characteres (< and >) to the words before computing character n-grams.

In [ ]:
def get_subwords(word: str, minn: int, maxn: int) -> List[str]:
    word = '<' + word + '>' 
    subwords = set(
        [
            word[i:i+size]
            for size in range(minn, maxn+1)
            for i in range(len(word)-size+1)
        ]
    )
    return list(subwords)

In [ ]:
get_subwords("meatballs", minn=minn, maxn=maxn)

['eatba',
 '<me',
 'meat',
 'tbal',
 'meatba',
 'tball',
 'atba',
 'tba',
 'bal',
 'alls',
 'eat',
 'ls>',
 'alls>',
 'balls',
 'eatb',
 '<mea',
 'ball',
 'tballs',
 'eatbal',
 'balls>',
 '<meatb',
 'all',
 'mea',
 'lls>',
 '<meat',
 'meatb',
 'atball',
 'atbal',
 'atb',
 'lls']

# 3. Find subword vectors in input matrix

The input matrix is of shape **(n_vocabulary + bucket_size, dim)**. For every unique word in the vocabulary there is a corresponding vector in the first chunk of the matrix. After this the remaining chunk (of bucket_size) holds vectors for subword and word n-grams, which might collide since the bucket size is fixed, as opposed to word vectors which do not collide.

To find the n-gram index in the matrix we have to do the following procedure:

1. Hash the subword string characters into a single integer.
2. Modulo this number with the bucket size, which will map the number in one of the bucket slots.
3. Add the vocabulary length to the previous result to get the index of the subword in the input_matrix.

In [ ]:
def get_hash(subword: str) -> int:
    h = 2166136261
    for c in subword:
        c = ord(c) % 2**8
        h = (h ^ c) % 2**32
        h = (h * 16777619) % 2**32
    return h

def get_subword_index(subword, bucket, nb_words):
    return (get_hash(subword) % bucket) + nb_words

In [ ]:
# Equivalent implementation
def get_hash(subword: str) -> np.uint32:
    h = np.uint32(2166136261)
    for c in subword:
        c = np.uint32(np.int8(ord(c)))
        h = np.uint32(h ^ c)
        h = np.uint32(h * 16777619)
    return h

In [ ]:
idx_a = get_subword_index('<me', bucket_size, len(vocabulary))
idx_a

16148

In [ ]:
idx_b = model.get_subword_id('<me')
idx_b

16148

In [ ]:
input_matrix[idx_a]

array([ 0.15881912, -2.008248  , -2.0042837 , -0.27310666, -1.9667645 ,
       -1.5717009 ,  1.1889542 , -0.9687559 ,  1.5461129 , -0.14315978,
        0.75167054, -0.01499958,  0.61466074, -0.5986607 ,  0.49294454,
       -0.610307  ], dtype=float32)

# 4. Find wordNgram features in the input matrix

To compute the wordNgrams for a particular sentence you need to: 
1. Get the hashes for the words in the sentence, including the EOS token.
2. Make ngrams from size 2 up to the max ngram size (**wordNgrams** parameter)
3. Get a single index (feature) per ngram by recursively hashing (again but different hashing function) each word hash integer into a single integer.

Suppose we want to get the wordNgram features for the sentence: **what is pizza?**

### 4.1. Get hashes for words in the sentence
In this case we will get the hashes for the following words:
1. what
2. is
3. pizza?
4. <\/s>

To do this we will use the previously defined `get_hash` function for characters.

In [ ]:
sentence = "what is pizza?"
# Append EOL token
sentence += " </s>"
# Get hashes for each word
words = sentence.split()
hashes = [np.int32(get_hash(word)) for word in words]


hashes, words

([-2056350673, 1312329493, 378301358, -677604519],
 ['what', 'is', 'pizza?', '</s>'])

### 4.2. Make ngrams from size 2 up to the max ngram size (wordNgrams parameter)

Now we have to generate the word ngrams (only if wordNgrams > 1). Word Ngrams are generated for each length from 2 up to the wordNgrams parameter or \[2, wordNgrams+1\].

As we already have a list of hashes instead of a sentence with words, we generate the ngrams using the hashes directly instead of doing it with words and then hashing.
  

In [ ]:
wordngrams = [
    words[i:i+size]
    for size in range(2, wordNgrams+1)
    for i in range(len(words)-size+1)
]
wordngrams

[['what', 'is'], ['is', 'pizza?'], ['pizza?', '</s>']]

In [ ]:
wordngrams = [
    hashes[i:i+size]
    for size in range(2, wordNgrams+1)
    for i in range(len(hashes)-size+1)
]
wordngrams

[[-2056350673, 1312329493], [1312329493, 378301358], [378301358, -677604519]]

### 4.3 Get a single index per ngram by recursively hashing each word hash integer into a single integer.

We want to combine each ngram's hashes (words) into a single index for the input_matrix. A different recursive hashing function is used than the one for hashing string characters. After hashes are obtained we pass through modulo % buckets and add the vocabulary length to find the actual index in the input matrix.

In [ ]:
wordngrams_indexes = list()

for ngram in wordngrams:
    h = ngram[0]
    for word_hash in ngram[1:]:
        h = np.uint64(h * 116049371 + word_hash)
    ngram_index = len(vocabulary) + int(h) % bucket_size
    wordngrams_indexes.append(ngram_index)
wordngrams_indexes

[18969, 114804, 155842]

We end up with the indices corresponding to the following 3 wordNgram features.

1. "what is"
2. "is pizza" 
3. "pizza <\/s>"

It's not as easy to verify the right index for a given wordngram using the Python model as it is with words and subwords, thanks to **model.get_word_id()** and **model.get_subword_id()**, there is not a **model.get_wordngram_id()** available to verify. So to check these are in fact the right indexes we have to modify the original source to print them which is not included in this notebook.

Once we have the wordNgram indexes we can look them up in the input matrix to retrieve the embeddings.

In [ ]:
features = [input_matrix[word_ngram_id] for word_ngram_id in wordngrams_indexes]
features

[array([-0.5212617 , -0.15074034, -0.80975354, -0.76292604, -0.16668485,
         1.5814468 , -0.87663573, -0.2633228 ,  0.16226742, -0.5250907 ,
         0.26750985, -0.05091313,  0.40993208,  0.6963279 , -0.5980508 ,
        -1.5497122 ], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([-0.4779425 ,  0.1734612 ,  0.46179655, -0.07976016,  0.30360892,
        -0.28149533,  0.42330447, -0.22358117, -0.61802363, -0.10678729,
        -0.01147463,  0.13381556,  1.1937575 ,  0.8168405 ,  0.01947787,
         0.72266304], dtype=float32)]

### Actual FastText implementation

The previous implementantion makes it easier to understand the steps involved, however in the original FastText C++ source, getting the wordNgram indexes for a given sentence is computed by generating and hashing the ngrams to a single list in one go. The following definition is a closer replica of the FastText process.

In [ ]:
def add_word_ngram(line: list, hashes: list, n: int, bucket: int, nwords: int) -> List[int]:
    for i in range(len(hashes)):
        h = np.int32(hashes[i])
        j = i + 1
        while j < len(hashes) and j < i + n:
            h = np.uint64(h * 116049371 + np.int32(hashes[j]))
            line.append(nwords + (int(h) % bucket))
            j += 1
    return line

In [ ]:
word_ngrams_idxs = add_word_ngram(line=list(), hashes=hashes, n=wordNgrams, bucket=bucket_size, nwords=len(vocabulary))
word_ngrams_idxs

[18969, 114804, 155842]

# 5. Replicating FastText vector functions

## 5.1 model.get_word_vector(string)

In the case of minn and maxn equal to zero (no character n-grams), the full word index is the final word vector. However, when we use character n-grams the final word vector is the mean of the full word vector and the compounding subword vectors. Notice that when computing sentence vectors we do not make use of model.get_word_vector(), since averaging all features at once (subwords, words, wordngrams) might produce a different result that if we average at word level and then average those again for a sentence.

In [ ]:
def get_word_vector(word: str) -> np.ndarray:
    subwords = get_subwords(word, minn, maxn)
    subword_idxs = [get_subword_index(subword, bucket_size, len(vocabulary)) for subword in subwords]
    embeddings = [input_matrix[idx] for idx in subword_idxs]
    if word in vocabulary:
        word_idx = vocabulary.index(word)
        embeddings += [input_matrix[word_idx]]
    return np.mean(embeddings, axis=0)

In [ ]:
get_word_vector("meatballs")

array([ 0.6272257 ,  0.09336659, -0.27236226,  0.40666848, -0.59998316,
        0.13910004,  1.4206132 , -0.4222818 ,  0.2345608 , -0.67761075,
        0.09557244, -0.32499897, -0.12342142,  0.38488638,  0.21018639,
       -0.03817398], dtype=float32)

In [ ]:
model.get_word_vector("meatballs")

array([ 0.62722564,  0.09336659, -0.27236217,  0.40666845, -0.5999832 ,
        0.13910004,  1.4206134 , -0.4222818 ,  0.23456082, -0.67761075,
        0.09557243, -0.32499892, -0.12342141,  0.38488638,  0.21018638,
       -0.03817399], dtype=float32)

In [ ]:
np.isclose(
    get_word_vector("meatballs"),
    model.get_word_vector("meatballs")
).all()

True

## 5.2 model.get_sentence_vector(string)

The sentence vector is the mean of all feature embeddings, these are all the individual full word embeddings, all the subword embeddings and all wordNgram embeddings. It also includes the End Of Sentence token as if it were a single word.

In [ ]:
def get_word_features(words: List[str], input_matrix: np.ndarray, vocabulary: List[str]):
    """
    Returns a list of all individual word features, 1 per word if word is in vocabulary
    """
    features = [input_matrix[vocabulary.index(word)] for word in words if word in vocabulary]
    return features


def get_subword_features(words: List[str], input_matrix: np.ndarray, vocabulary: List[str], minn: int, maxn: int, bucket_size: int) -> List[np.ndarray]:
    """
    Returns a list of all subword features, all the variable number of subword features
    from each word are flattened together in a single list
    """
    
    def get_subwords(word: str, minn: int, maxn: int) -> List[str]:
        word = '<' + word + '>' 
        subwords = set(
            [
                word[i:i+size]
                for size in range(minn, maxn+1)
                for i in range(len(word)-size+1)
            ]
        )
        return list(subwords)

    def get_hash(subword: str) -> int:
        h = 2166136261
        for c in subword:
            c = ord(c) % 2**8
            h = (h ^ c) % 2**32
            h = (h * 16777619) % 2**32
        return h

    def get_subword_index(subword: str, bucket: int, nb_words: int):
        return (get_hash(subword) % bucket) + nb_words

    
    features = list()
    for word in words:
        if word == "</s>":
          continue
        subwords = get_subwords(word, minn, maxn)
        subword_idxs = [get_subword_index(subword, bucket_size, len(vocabulary)) for subword in subwords]
        subword_embeddings = [input_matrix[idx] for idx in subword_idxs]
        features += subword_embeddings
    return features


def get_wordNgram_features(words: List[str], input_matrix: np.ndarray, nwords: int, max_ngram_length: int, bucket: int) -> List[np.ndarray]:
    """
    Returns a list of all wordNgram features according to size n
    """
    def get_hash(subword: str) -> int:
        h = 2166136261
        for c in subword:
            c = ord(c) % 2**8
            h = (h ^ c) % 2**32
            h = (h * 16777619) % 2**32
        return h
  
    def get_wordngram_index(hashes: list, bucket: int, nwords: int) -> int:
        h = hashes[0]
        for word_hash in hashes[1:]:
            h = np.uint64(h * 116049371 + word_hash)
        return (nwords + int(h) % bucket)

    def get_wordngrams(words: List[str], max_ngram_length: int) -> List[np.ndarray]:
        hashes = [np.int32(get_hash(word)) for word in words]
        wordngrams = [
            hashes[i:i+size]
            for size in range(2, max_ngram_length+1)
            for i in range(len(hashes)-size+1)
        ]
        return wordngrams

    ngrams = get_wordngrams(words, max_ngram_length)
    word_ngrams_idxs = [get_wordngram_index(ngram, bucket, nwords) for ngram in ngrams]
    features = [input_matrix[word_ngram_id] for word_ngram_id in word_ngrams_idxs]
    
    return features

In [ ]:
def get_sentence_vector(sentence: str, minn: int, maxn: int, wordNgrams: int, bucket_size: int, vocabulary: List[str], input_matrix: np.ndarray) -> np.ndarray:
    """
    Equivalent to model.get_sentence_vector()
    """
    sentence += " </s>"
    words = sentence.split()
    feature_embeddings = []
    
    # Get word features
    feature_embeddings += get_word_features(words, input_matrix, vocabulary)
    
    # Get subword features
    if minn > 0 and maxn > 0:
        feature_embeddings += get_subword_features(words, input_matrix, vocabulary, minn, maxn, bucket_size)

    # Get wordNgram features
    if wordNgrams > 1:
        feature_embeddings += get_wordNgram_features(words, input_matrix, len(vocabulary), 
                                                     wordNgrams, bucket_size)
    
    # Compute mean of all features
    sentence_vec = np.mean(feature_embeddings, axis=0)
    return sentence_vec

In [ ]:
# Compare against model generated sentence vector
sentence = "How to make a pepperoni pizza?"
np.isclose(
    get_sentence_vector(sentence, minn, maxn, wordNgrams, bucket_size, vocabulary, input_matrix),
    model.get_sentence_vector(sentence)
).all()

True

In [ ]:
get_sentence_vector(sentence, minn, maxn, wordNgrams, bucket_size, vocabulary, input_matrix)

array([-0.03875927,  0.05721441,  0.19570303,  0.00276239,  0.25914463,
       -0.51486284, -0.09010237,  0.0232471 , -0.2304556 , -0.02440632,
        0.17151904, -0.05499319,  0.07283257,  0.3178859 ,  0.15399942,
       -0.19516972], dtype=float32)

In [ ]:
model.get_sentence_vector(sentence)

array([-0.03875926,  0.05721441,  0.1957031 ,  0.00276241,  0.25914463,
       -0.5148627 , -0.09010238,  0.02324709, -0.2304556 , -0.02440632,
        0.171519  , -0.05499319,  0.07283266,  0.3178859 ,  0.15399945,
       -0.19516975], dtype=float32)

# 5. Compute predictions using output_matrix

**\*Assumes model uses a softmax loss**

The supervised model FastText uses is a single layer neural network. Once we have a sentence vector we can obtain the output class vector by matrix multiplying the output_matrix (n_classes, dim) and the sentence vector (dim,), which will result in an output vector with shape (n_classes,) which has a value for each possible class. These scores are then softmaxed to convert to probabilities and then the highest one is selected for prediction.

In this notebook we are assuming that we are using the default softmax loss when training the supervised model. To implement a different loss function we need to define it as a function and apply it to the output vector instead of **softmax** before selecting the prediction.



In [ ]:
output_matrix.shape

(735, 16)

In [ ]:
def softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

In [ ]:
def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

In [ ]:
def get_prediction(sentence: str, minn: int, maxn: int, wordNgrams: int, bucket_size: int, vocabulary: List[str], input_matrix: np.ndarray):
    sentence_vec = get_sentence_vector(sentence, minn, maxn, wordNgrams, bucket_size, vocabulary, input_matrix)
    pred_idx = np.argmax(softmax(np.matmul(output_matrix, sentence_vec)))
    pred_proba = np.max(softmax(np.matmul(output_matrix, sentence_vec)))
    pred_label = labels[pred_idx]
    return pred_label, pred_proba

In [ ]:
get_prediction("Italian food recipes", minn, maxn, wordNgrams, bucket_size, vocabulary, input_matrix)

('__label__baking', 0.6910467)

In [ ]:
model.predict("Italian food recipes")

(('__label__baking',), array([0.6910578]))

In [ ]:
np.isclose(get_sentence_vector(sentence, minn, maxn, wordNgrams, bucket_size, vocabulary, input_matrix), model.get_sentence_vector(sentence)).all()

True